# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [134]:
import pandas as pd
import numpy as np
import pickle

Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

In [133]:
# El archivo estaba comprimido. Hay que descomprimirlo primero
loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', compression = 'gzip', low_memory = False)

loans

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,2,37842129,NaN,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,3,37612354,NaN,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,4,37662224,NaN,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,22454240,NaN,8400.0,8400.0,8400.0,36 months,9.17,267.79,B,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99996,99996,11396920,NaN,10000.0,10000.0,10000.0,36 months,12.99,336.90,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99997,99997,8556176,NaN,30000.0,30000.0,30000.0,60 months,20.99,811.44,E,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99998,99998,24023408,NaN,8475.0,8475.0,8475.0,36 months,24.99,336.92,F,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [21]:
column_types = loans.dtypes
column_types

Unnamed: 0                 int64
id                         int64
member_id                float64
loan_amnt                float64
funded_amnt              float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 151, dtype: object

## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [22]:


datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']


In [23]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [24]:
# Codigo para 
# Como el pickle no es human-readable, se guarda en binario. Por eso se usa 'wb' (write-binary)
with open('datos_dict.pkl', 'wb') as file:
    pickle.dump(datos_dict, file)

In [25]:
# Codigo para cargar
datos_dict_from_pickle = None
# 'rb' = Read-binary. La razón es la misma que más arriba
with open('datos_dict.pkl', 'rb') as file:
    datos_dict_from_pickle = pickle.load(file)
print(datos_dict_from_pickle)

                   feature                                        description
0           acc_now_delinq  The number of accounts on which the borrower i...
1     acc_open_past_24mths         Number of trades opened in past 24 months.
2               addr_state  The state provided by the borrower in the loan...
3                 all_util              Balance to credit limit on all trades
4               annual_inc  The self-reported annual income provided by th...
..                     ...                                                ...
148      settlement_amount  The loan amount that the borrower has agreed t...
149  settlement_percentage  The settlement amount as a percentage of the p...
150        settlement_term  The number of months that the borrower will be...
151                    NaN                                                NaN
152                    NaN  * Employer Title replaces Employer Name for al...

[153 rows x 2 columns]


## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [135]:
# Manejos de tipos 1: Fechas
# Todas las fechas del documento pueden tener el mismo formato: Mes-año. Hay que normalizar para poder usar formato de fecha

# Empezamos por una lista de los nombres de las columnas que contienen fechas
columnas_fechas = ["earliest_cr_line", "last_pymnt_d", "next_pymnt_d", "hardship_start_date", 
                   "hardship_end_date", "payment_plan_start_date", "settlement_date", "debt_settlement_flag_date",]


In [136]:
# Ahora hacemos el proceso de normalización:
for columna in columnas_fechas:
    # Cambiamos los strings a letras minusculas
    loans[columna] = loans[columna].str.lower()
    
    # Detectamos el formato de fecha y cambiamos el tipo de dato. Así aparecerá en formato aaaa-mm-dd
    loans[columna] = loans[columna].apply(lambda x: pd.to_datetime(x, format='%b-%y' if len(str(x)) == 6 else '%b-%Y', errors='coerce'))
    
    # Cambiamos el formato para que aparezca como aaaa-mm, sin día
    loans[columna] = loans[columna].apply(lambda x: x.to_period('M') if not pd.isnull(x) else x)

# Verificar el resultado
print(loans[columnas_fechas])

# Con esto quedó guardado en formato de fecha, específicamente period[M]

      earliest_cr_line last_pymnt_d next_pymnt_d hardship_start_date  \
0              1994-08      2016-06          NaT                 NaT   
1              1989-09      2016-08          NaT                 NaT   
2              2003-08      2016-05          NaT                 NaT   
3              2000-10      2017-12      2018-01                 NaT   
4              2002-08      2015-08          NaT                 NaT   
...                ...          ...          ...                 ...   
99995          1990-08      2017-08          NaT                 NaT   
99996          2001-05      2016-03          NaT                 NaT   
99997          1998-06      2017-12      2018-01                 NaT   
99998          2000-11      2015-04          NaT                 NaT   
99999          1999-07      2017-03          NaT                 NaT   

      hardship_end_date payment_plan_start_date settlement_date  \
0                   NaT                     NaT             NaT   
1

In [138]:
column_types = loans.dtypes
column_types

Unnamed: 0                   int64
id                           int64
member_id                  float64
loan_amnt                  float64
funded_amnt                float64
                           ...    
settlement_status           object
settlement_date          period[M]
settlement_amount          float64
settlement_percentage      float64
settlement_term            float64
Length: 151, dtype: object

In [137]:
prueba = loans.copy()
prueba

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,...,NaN,NaN,Cash,N,NaT,NaN,NaT,NaN,NaN,NaN
1,1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,...,NaN,NaN,Cash,N,NaT,NaN,NaT,NaN,NaN,NaN
2,2,37842129,NaN,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,...,NaN,NaN,Cash,N,NaT,NaN,NaT,NaN,NaN,NaN
3,3,37612354,NaN,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,...,NaN,NaN,Cash,N,NaT,NaN,NaT,NaN,NaN,NaN
4,4,37662224,NaN,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,...,NaN,NaN,Cash,N,NaT,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,22454240,NaN,8400.0,8400.0,8400.0,36 months,9.17,267.79,B,...,NaN,NaN,Cash,N,NaT,NaN,NaT,NaN,NaN,NaN
99996,99996,11396920,NaN,10000.0,10000.0,10000.0,36 months,12.99,336.90,C,...,NaN,NaN,Cash,N,NaT,NaN,NaT,NaN,NaN,NaN
99997,99997,8556176,NaN,30000.0,30000.0,30000.0,60 months,20.99,811.44,E,...,NaN,NaN,Cash,N,NaT,NaN,NaT,NaN,NaN,NaN
99998,99998,24023408,NaN,8475.0,8475.0,8475.0,36 months,24.99,336.92,F,...,NaN,NaN,Cash,N,NaT,NaN,NaT,NaN,NaN,NaN


In [149]:
loans["chargeoff_within_12_mths"]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
99995    0.0
99996    0.0
99997    0.0
99998    0.0
99999    0.0
Name: chargeoff_within_12_mths, Length: 100000, dtype: float64

In [ ]:
# Manejos de tipos 2: Enteros
'''
    La mayoría de los datos que podemos convertir a enteros tienen una característica: la descripcio de la columna 
    refiere a numeros enteros no negativos y no convienen NaN's. Hay algunas columnas que si refieren a numeros enteros pero que
    al contener Nan's no se puede hacer el casteo a int64.
'''

# Los datos que podemos castear directamente a enteros (por su descripción y por cómo vienen en el csv) son los siguientes:
columnas_enteros = ["delinq_2yrs", "inq_last_6mths", "open_acc", "pub_rec", "collections_12_mths_ex_med", "policy_code", "acc_now_delinq", "tot_coll_amt", "acc_open_past_24mths", "chargeoff_within_12_mths",]

'''
Consideraciones especiales:
    'policy_code' solo puede tomar los valores 1 o 2. Lo cambie a int porque ocupa menos espacio en memoria que un object o que un float
'''



In [ ]:
# column_types =


## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [ ]:
# Tu codigo aqui

In [ ]:
# TUcodigo aqui

### Codigo para salvar y cargar JSONs